In [20]:
import psycopg2
import pandas as pd
from pathlib import Path
import csv


conn = psycopg2.connect(database="f1_halo",
                       host="localhost",
                       user="postgres",
                       password = "postgres",
                       port="5432")

In [21]:
# Store filepaths into variable
driver_csv = Path("data/drivers.csv")
fcyphases_csv = Path("data/fcyphases.csv")
laps_csv = Path("data/laps.csv")
qualifyings_csv = Path("data/qualifyings.csv")
races_csv = Path("data/races.csv")
retirements_csv = Path("data/retirements.csv")
starterfields_csv = Path("data/starterfields.csv")

In [22]:
tables = ["drivers","fcyphases","laps","qualifyings","races","retirements","starterfields"]
for i in tables:
    cursor = conn.cursor()
    query = f'SELECT * from {i};'
    #print(query)
    cursor.execute(query)
    rows = cursor.fetchall()
    df_name = i + "_df"
    globals()[df_name]= pd.DataFrame(rows)
    print(df_name)
    
    
    #csv for headers
    path = f'data/{i}.csv'
    csv_path = Path(path)
    with open(csv_path, 'r', newline='') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the first row
        if len(globals()[df_name].columns) == len(headers):
            globals()[df_name].columns = headers

    #print(df_name)
    print(globals()[df_name].columns)

drivers_df
Index(['id', 'carno', 'initials', 'name'], dtype='object')
fcyphases_df
Index(['id', 'race_id', 'startracetime', 'endracetime', 'startraceprog',
       'endraceprog', 'startlap', 'endlap', 'type'],
      dtype='object')
laps_df
Index(['race_id', 'lapno', 'position', 'driver_id', 'laptime', 'racetime',
       'gap', 'interval', 'compound', 'tireage', 'pitintime',
       'pitstopduration', 'nextcompound', 'startlapprog_vsc', 'endlapprog_vsc',
       'age_vsc', 'startlapprog_sc', 'endlapprog_sc', 'age_sc'],
      dtype='object')
qualifyings_df
Index(['race_id', 'position', 'driver_id', 'q1laptime', 'q2laptime',
       'q3laptime', 'speedtrap'],
      dtype='object')
races_df
Index(['id', 'date', 'season', 'location', 'availablecompounds', 'comment',
       'nolaps', 'nolapsplanned', 'tracklength'],
      dtype='object')
retirements_df
Index(['season', 'driver_id', 'accidents', 'failures'], dtype='object')
starterfields_df
Index(['race_id', 'driver_id', 'team', 'teamcolor', 'eng

In [23]:
fcyphases_df.head()

,id,race_id,startracetime,endracetime,startraceprog,endraceprog,startlap,endlap,type
0,1,1,1069,1604,11.107,14.922,12,15,SC
1,2,3,4157,4899,41.010,45.965,42,46,SC
2,3,6,59,332,0.583,2.897,1,3,SC
3,4,6,2157,2710,25.289,29.916,26,30,SC
4,5,7,38,829,0.372,6.864,1,7,SC


In [24]:
#races_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#races_df.head()

In [25]:
#starterfields_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#starterfields_df.head()

In [26]:
drivers_df.rename(columns = {'id':'driver_id'}, inplace = True)
drivers_df.head()
races_df.rename(columns = {'id':'race_id'}, inplace = True)
races_df.head()

,race_id,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
1,2,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",null,56,56,5543
2,3,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412
3,4,2014-04-20,2014,Shanghai,"A2,A3,I,W",null,54,56,5451
4,5,2014-05-11,2014,Catalunya,"A1,A2,I,W",null,66,66,4655


In [50]:
# make column name variables constant throughout all dfs
drivers_df.rename(columns = {'id':'driver_id'}, inplace = True)
# drivers_df.to_csv('drivers.csv') only do this if create a seperate data file
races_df.rename(columns = {'id':'race_id'}, inplace = True)
races_df

,race_id,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
1,2,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",null,56,56,5543
2,3,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412
3,4,2014-04-20,2014,Shanghai,"A2,A3,I,W",null,54,56,5451
4,5,2014-05-11,2014,Catalunya,"A1,A2,I,W",null,66,66,4655
...,...,...,...,...,...,...,...,...,...
116,117,2019-10-13,2019,Suzuka,"A2,A3,A4,I,W",null,52,53,5807
117,118,2019-10-27,2019,MexicoCity,"A3,A4,A6,I,W",null,71,71,4304
118,119,2019-11-03,2019,Austin,"A3,A4,A6,I,W",null,56,56,5513
119,120,2019-11-17,2019,SaoPaulo,"A2,A3,A4,I,W",null,71,71,4309


In [43]:
merge1 = pd.merge(drivers_df, starterfields_df)
merge1

,driver_id,carno,initials,name,race_id,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,completedlaps,speedtrap
0,1,44,HAM,Lewis Hamilton,1,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8
1,1,44,HAM,Lewis Hamilton,2,Mercedes,#00D2BE,Mercedes,1,F,1,56,305.9
2,1,44,HAM,Lewis Hamilton,3,Mercedes,#00D2BE,Mercedes,2,F,1,57,313.4
3,1,44,HAM,Lewis Hamilton,4,Mercedes,#00D2BE,Mercedes,1,F,1,54,332.3
4,1,44,HAM,Lewis Hamilton,5,Mercedes,#00D2BE,Mercedes,1,F,1,66,331.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,45,88,KUB,Robert Kubica,117,Williams,#192c4e,Mercedes,20,F,19,50,299.3
2475,45,88,KUB,Robert Kubica,118,Williams,#192c4e,Mercedes,20,F,18,69,352
2476,45,88,KUB,Robert Kubica,119,Williams,#192c4e,Mercedes,19,DNF,19,31,303.7
2477,45,88,KUB,Robert Kubica,120,Williams,#192c4e,Mercedes,19,F,16,70,313.5


In [51]:
merge2=pd.merge(newdf, races_df)
meta_df = pd.merge(merge2, retirements_df)
meta_df

,driver_id,carno,initials,name,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,...,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength,race_id,season,accidents,failures
0,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,Melbourne,"A2,A3,I,W",null,57,58,5303,1,2014,0,3
1,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,F,1,...,KualaLumpur,"A1,A2,I,W",null,56,56,5543,2,2014,0,3
2,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,2,F,1,...,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412,3,2014,0,3
3,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,F,1,...,Shanghai,"A2,A3,I,W",null,54,56,5451,4,2014,0,3
4,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,F,1,...,Catalunya,"A1,A2,I,W",null,66,66,4655,5,2014,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,20,F,19,...,Suzuka,"A2,A3,A4,I,W",null,52,53,5807,117,2019,0,2
2475,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,20,F,18,...,MexicoCity,"A3,A4,A6,I,W",null,71,71,4304,118,2019,0,2
2476,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,DNF,19,...,Austin,"A3,A4,A6,I,W",null,56,56,5513,119,2019,0,2
2477,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,16,...,SaoPaulo,"A2,A3,A4,I,W",null,71,71,4309,120,2019,0,2


In [45]:
meta_df_refined = meta_df[['season', 'date', 'race_id','location','driver_id','name','team','enginemanufacturer','resultposition','gridposition','status','completedlaps','comment','nolaps','tracklength']]
meta_df_refined = meta_df_refined.set_index(['season'])
meta_df_refined

,date,race_id,location,driver_id,name,team,enginemanufacturer,resultposition,gridposition,status,completedlaps,comment,nolaps,tracklength
season,,,,,,,,,,,,,,
2014,2014-03-16,1,Melbourne,1,Lewis Hamilton,Mercedes,Mercedes,19,1,DNF,2,null,57,5303
2014,2014-03-30,2,KualaLumpur,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,56,null,56,5543
2014,2014-04-06,3,Sakhir,1,Lewis Hamilton,Mercedes,Mercedes,1,2,F,57,"Crash GUT in lap 41, SC somewhen in the end of...",57,5412
2014,2014-04-20,4,Shanghai,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,54,null,54,5451
2014,2014-05-11,5,Catalunya,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,66,null,66,4655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019-10-13,117,Suzuka,45,Robert Kubica,Williams,Mercedes,19,20,F,50,null,52,5807
2019,2019-10-27,118,MexicoCity,45,Robert Kubica,Williams,Mercedes,18,20,F,69,null,71,4304
2019,2019-11-03,119,Austin,45,Robert Kubica,Williams,Mercedes,19,19,DNF,31,null,56,5513


In [47]:
meta_df=pd.merge(newdf, retirements_df)
meta_df

,driver_id,carno,initials,name,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,...,date,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength,season,accidents,failures
0,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2014,0,3
1,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2015,0,1
2,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2016,1,1
3,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2017,0,0
4,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2018,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14869,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2015,null,null
14870,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2016,null,null
14871,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2017,null,null
14872,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2018,null,null


In [52]:
meta_df_refined = meta_df[['season', 'date', 'race_id','location','driver_id','name','team','enginemanufacturer','resultposition','gridposition','status','completedlaps','comment','nolaps','tracklength']]
meta_df_refined = meta_df_refined.set_index(['season'])
meta_df_refined

,date,race_id,location,driver_id,name,team,enginemanufacturer,resultposition,gridposition,status,completedlaps,comment,nolaps,tracklength
season,,,,,,,,,,,,,,
2014,2014-03-16,1,Melbourne,1,Lewis Hamilton,Mercedes,Mercedes,19,1,DNF,2,null,57,5303
2014,2014-03-30,2,KualaLumpur,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,56,null,56,5543
2014,2014-04-06,3,Sakhir,1,Lewis Hamilton,Mercedes,Mercedes,1,2,F,57,"Crash GUT in lap 41, SC somewhen in the end of...",57,5412
2014,2014-04-20,4,Shanghai,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,54,null,54,5451
2014,2014-05-11,5,Catalunya,1,Lewis Hamilton,Mercedes,Mercedes,1,1,F,66,null,66,4655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019-10-13,117,Suzuka,45,Robert Kubica,Williams,Mercedes,19,20,F,50,null,52,5807
2019,2019-10-27,118,MexicoCity,45,Robert Kubica,Williams,Mercedes,18,20,F,69,null,71,4304
2019,2019-11-03,119,Austin,45,Robert Kubica,Williams,Mercedes,19,19,DNF,31,null,56,5513


In [53]:
meta_df=meta_df.set_index(['season','date'])
meta_df

driver_id  carno initials            name      team  \
season date                                                              
2014   2014-03-16          1     44      HAM  Lewis Hamilton  Mercedes   
       2014-03-30          1     44      HAM  Lewis Hamilton  Mercedes   
       2014-04-06          1     44      HAM  Lewis Hamilton  Mercedes   
       2014-04-20          1     44      HAM  Lewis Hamilton  Mercedes   
       2014-05-11          1     44      HAM  Lewis Hamilton  Mercedes   
...                      ...    ...      ...             ...       ...   
2019   2019-10-13         45     88      KUB   Robert Kubica  Williams   
       2019-10-27         45     88      KUB   Robert Kubica  Williams   
       2019-11-03         45     88      KUB   Robert Kubica  Williams   
       2019-11-17         45     88      KUB   Robert Kubica  Williams   
       2019-12-01         45     88      KUB   Robert Kubica  Williams   

                  teamcolor enginemanufacturer  gridposition status  \
season date                                                           
2014   2014-03-16   #00D2BE           Mercedes             1    DNF   
       2014-03-30   #00D2BE           Mercedes             1      F   
       2014-04-06   #00D2BE           Mercedes             2      F   
       2014-04-20   #00D2BE           Mercedes             1      F   
       2014-05-11   #00D2BE           Mercedes             1      F   
...                     ...                ...           ...    ...   
2019   2019-10-13   #192c4e           Mercedes            20      F   
       2019-10-27   #192c4e           Mercedes            20      F   
       2019-11-03   #192c4e           Mercedes            19    DNF   
       2019-11-17   #192c4e           Mercedes            19      F   
       2019-12-01   #192c4e           Mercedes            19      F   

                   resultposition  ...  speedtrap     location  \
season date                        ...                           
2014   2014-03-16              19  ...      252.8    Melbourne   
       2014-03-30               1  ...      305.9  KualaLumpur   
       2014-04-06               1  ...      313.4       Sakhir   
       2014-04-20               1  ...      332.3     Shanghai   
       2014-05-11               1  ...      331.5    Catalunya   
...                           ...  ...        ...          ...   
2019   2019-10-13              19  ...      299.3       Suzuka   
       2019-10-27              18  ...        352   MexicoCity   
       2019-11-03              19  ...      303.7       Austin   
       2019-11-17              16  ...      313.5     SaoPaulo   
       2019-12-01              19  ...      317.8    YasMarina   

                  availablecompounds  \
season date                            
2014   2014-03-16          A2,A3,I,W   
       2014-03-30          A1,A2,I,W   
       2014-04-06          A2,A3,I,W   
       2014-04-20          A2,A3,I,W   
       2014-05-11          A1,A2,I,W   
...                              ...   
2019   2019-10-13       A2,A3,A4,I,W   
       2019-10-27       A3,A4,A6,I,W   
       2019-11-03       A3,A4,A6,I,W   
       2019-11-17       A2,A3,A4,I,W   
       2019-12-01       A4,A6,A7,I,W   

                                                             comment nolaps  \
season date                                                                   
2014   2014-03-16                                               null     57   
       2014-03-30                                               null     56   
       2014-04-06  Crash GUT in lap 41, SC somewhen in the end of...     57   
       2014-04-20                                               null     54   
       2014-05-11                                               null     66   
...                                                              ...    ...   
2019   2019-10-13                                               null     52   
       2019-10-27                        

In [54]:
# save meta data frame to file in data folder
meta_df.to_csv('meta_data.csv')
meta_df_refined.to_csv('meta_data_refined.csv')